# SAMap preprocess
With leiden-based neighborhoods

In [ ]:
from samap.mapping import SAMAP
from samalg import SAM
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import csv
import warnings
warnings.filterwarnings(action='ignore')
import pickle

def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

## Set parameters

In [ ]:
# species A
sp_stage_A='AmpG4'
abbr_A='bf' # must be equal to filename under blast map folder
h5ad_A='Amp_G4.h5ad'

# species B
sp_stage_B='NV24hr'
abbr_B='nv' # must be equal to filename under blast map folder
h5ad_B='Gastrula24hrFINAL.h5ad'

# blast maps and id maps
g_map_folder='./maps_prot/'

## Read gene name mapping

Parse mapping of transcript ids vs gene names used in the anndata object

In [ ]:
with open(g_map_folder+'/'+abbr_A+'_idmap.csv') as f:
    idmap_A=[tuple(line) for line in csv.reader(f)][2:]

with open(g_map_folder+'/'+abbr_B+'_idmap.csv') as f:
    idmap_B=[tuple(line) for line in csv.reader(f)][2:]

print(abbr_A+" first five gene ids:")
print(idmap_A[:5])
print(abbr_B+" first five gene ids:")
print(idmap_B[:5])

# Running SAMap

## Loading in raw data

Instantiate the SAMAP object.

In [ ]:
sm = SAMAP(
        {abbr_A : h5ad_A , abbr_B : h5ad_B},
        f_maps = g_map_folder,
        names = {abbr_A : idmap_A , abbr_B : idmap_B},
        save_processed=False
    )

Show unintegrated umaps.

In [ ]:
sc.pl.embedding(sm.sams[abbr_A].adata,
                    basis='X_umap',
                    color='celltype_fine',
                title=(sp_stage_A+' Unintegrated'),
                    frameon=False)

sc.pl.embedding(sm.sams[abbr_B].adata,
                    basis='X_umap',
                title=(sp_stage_B+' Unintegrated'),
                    color='cell_type',
                    frameon=False)

## Run SAMap

In [ ]:
sm.run(ncpus=4)

Save samap object.

In [ ]:
save_object(sm,'sam_processed.pkl')

## Visualizing the combined projection

In [ ]:
fig, ax = plt.subplots()
fig.set_figwidth(5)
fig.set_figheight(5)
fig.tight_layout()
ax.set_title('SAMap UMAP '+sp_stage_A+' (red) vs '+sp_stage_B+' (blue)')
sm.scatter(COLORS={abbr_B:'#10b9f1',abbr_A:'#f14810'},axes=ax,ss={abbr_A:5,abbr_B:5},alpha=0.5)

# Save parameters

In [ ]:
params_d={
'sp_stage_A' : sp_stage_A,
'abbr_A' : abbr_A,
'h5ad_A' : h5ad_A,
'sp_stage_B' : sp_stage_B,
'abbr_B' : abbr_B,
'h5ad_B' : h5ad_B,
'g_map_folder' : g_map_folder}

pd.DataFrame.from_dict(data=params_d, orient='index').to_csv('samap_parameters.csv', header=False)